In this we are going to Understand How we Build Training Pipeline in PyTorch.

1. Load the dataset
2. Basic preprocessing (Scaling, Encoding)
3. Training Process
  - Create the model
  - Forward pass
  - Loss calculation
  - Backprop
  - Parameters update (using Gradient Descent)
4. Model evaluation using (Accuracy)


In [105]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [106]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [107]:
df.shape

(569, 33)

## **PreProcessing**

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [109]:
# Drop Unnecessary columns
df.drop(columns=['id','Unnamed: 32'], inplace=True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [110]:
df.shape

(569, 31)

**Train Test Split**

Here our first column is target column.

In [111]:
x_train, x_test, y_train , y_test = train_test_split(df.iloc[:, 1:], df.iloc[:,0], test_size=0.2)

Now Most of Our Values in Multiple Columns are not in a standard form. That means Every Columns has its own range of values highs and lows and not all values are in a uniform scale. This will be tough for comparision. Thus we have to make them into some standard unit.

In [112]:
# Scaling
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [113]:
x_train

array([[-2.60290307e-01,  1.59112874e-01, -2.19330964e-01, ...,
         3.10251263e-01, -1.01218087e-01,  2.22339218e+00],
       [-1.64459945e-01, -3.62297552e-01, -2.44369184e-01, ...,
        -9.42359161e-01, -8.98112204e-01, -1.15670484e+00],
       [ 2.55164631e+00,  1.10175006e+00,  2.49234801e+00, ...,
         1.40376448e+00, -4.94977533e-01, -3.56055553e-01],
       ...,
       [ 1.43727610e+00, -1.39161213e-01,  1.41928144e+00, ...,
         1.36044223e+00,  5.56610193e-01,  3.33859867e-01],
       [-6.51825785e-01, -4.64758116e-01, -6.80352158e-01, ...,
        -1.07367038e+00,  4.53482719e-01, -9.86804348e-01],
       [-1.01485707e-01,  1.15184190e+00, -2.33305724e-03, ...,
         5.55246027e-01,  1.15818712e+00,  1.86513555e+00]])

In [114]:
x_train.dtype

dtype('float64')

In [115]:
x_test

array([[ 0.62750883, -1.2470153 ,  0.57707734, ...,  0.39927373,
        -0.27242329, -0.80595028],
       [ 0.54204536,  0.19642886,  0.56228803, ...,  0.63128538,
        -0.05332477, -0.22980389],
       [-0.05619898, -0.01163516, -0.07710336, ...,  0.25012339,
        -0.03026176,  0.38523845],
       ...,
       [ 1.4206099 , -0.48758161,  1.34119195, ...,  0.66608713,
        -0.04371518, -0.10460752],
       [ 2.1555958 ,  0.21203367,  2.14474473, ...,  1.72339736,
         0.32144904, -0.19576993],
       [-0.8971596 , -1.02594728, -0.93241874, ..., -1.6014953 ,
        -0.09560694, -0.59506125]])

In [116]:
x_test.dtype

dtype('float64')

**Now our Y has 2 categories i.e Not Numbers. Thus we have to encode it into Numbers and we will do this using Label Encoder.Because it has 2 categories so Binary Classification Problem**

In [117]:
y_train

,diagnosis
465,B
278,B
339,M
381,B
485,B
...,...
68,B
195,B
487,M
188,B


In [118]:
y_test

,diagnosis
133,B
131,M
523,B
316,B
138,M
...,...
264,M
497,B
444,M
432,M


In [119]:
# Encode it to Numbers

encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [120]:
y_train

array([0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,

In [121]:
y_train.dtype

dtype('int64')

**Convert the np.array to Tensors**

In [122]:
x_train_tensor = torch.from_numpy(x_train.astype(np.float32))
x_test_tensor = torch.from_numpy(x_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [123]:
x_train_tensor

tensor([[-2.6029e-01,  1.5911e-01, -2.1933e-01,  ...,  3.1025e-01,
         -1.0122e-01,  2.2234e+00],
        [-1.6446e-01, -3.6230e-01, -2.4437e-01,  ..., -9.4236e-01,
         -8.9811e-01, -1.1567e+00],
        [ 2.5516e+00,  1.1018e+00,  2.4923e+00,  ...,  1.4038e+00,
         -4.9498e-01, -3.5606e-01],
        ...,
        [ 1.4373e+00, -1.3916e-01,  1.4193e+00,  ...,  1.3604e+00,
          5.5661e-01,  3.3386e-01],
        [-6.5183e-01, -4.6476e-01, -6.8035e-01,  ..., -1.0737e+00,
          4.5348e-01, -9.8680e-01],
        [-1.0149e-01,  1.1518e+00, -2.3331e-03,  ...,  5.5525e-01,
          1.1582e+00,  1.8651e+00]])

In [124]:
x_train_tensor.shape

torch.Size([455, 30])

In [125]:
x_train_tensor.dtype

torch.float32

## **Model Building(Define the Model)**

**Now changes will be Made Here**

- use nn.Module
- change constructor and forward pass function to use nn.module attributes.
- Use Built in Loss Function
- Dont calculate and update gradient manually use built in optimizations.
- Using Stocastic Gradient Descent. This requires all parameters and learning rate.

The `model.parameters()` method in PyTorch retrieves an iterator over all the trainable parameters (weights and biases) in a model. These parameters are instances of torch.nn.Parameter and include:

- **Weights**: The weight matrices of layers like nn.Linear, nn.Conv2d, etc.
- **Biases**: The bias terms of layers (if they exist).

The optimizer uses these parameters to compute gradients and update them during training.

In [126]:
import torch.nn as nn

In [127]:
"""
This is old code before nn Module & Loss Function. Just here for reference that how much we are cleaning.
"""

# class MySimpleNetwork():

#   # constructor function getting data
#   def __init__(self, X):
#     """
#     We have 30 input features thus we will need 30 weights for that.
#     So making a matrix of 30x1.
#     We need to update weights and perform all derivates on weights and bias thus we make grad=True.
#     """

#     self.weights = torch.rand(X.shape[1], 1 , dtype=torch.float64, requires_grad=True)

#     # we are making one neuron only thus one neuron with its bias as 0 in start
#     self.bias = torch.zeros(1 , dtype=torch.float64, requires_grad=True)


#   # Forward Pass
#   def forward_pass(self,X):

#     # Calculate z
#     z = torch.matmul(X,self.weights) + self.bias

#     # predict y with activation on z
#     y_pred = torch.sigmoid(z)

#     return y_pred

#   # Loss Function
#   def loss_function(self, y_pred, y):
#     """
#     Since we are doing binary classification.
#     We decide to choose Binary Cross Entropy as a Loss Function.
#     """
#     # Clamp predictions to avoid log(0)
#     epsilon = 1e-7
#     y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

#     # Calculate loss(Formula Based)
#     loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()

#     return loss

'\nThis is old code before nn Module & Loss Function. Just here for reference that how much we are cleaning.\n'

In [128]:
class MySimpleNetwork(nn.Module):

  # constructor function getting data
  def __init__(self, num_features):

    super().__init__()

    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()


  # Forward Pass
  def forward(self,features):

    # Calculate z
    z = self.linear(features)
    y_pred = self.sigmoid(z)

    return y_pred

**Important Parameter**

In [129]:
# We need to define the learning rates which are usefull for optimization.
learning_rate = 0.1

# How many runs/loops on data
epochs = 25

In [130]:
# Loss Function
loss_function = nn.BCELoss()

In [131]:
loss_function

BCELoss()

In [132]:
type(loss_function)

torch.nn.modules.loss.BCELoss

## **Training Pipeline**

**Create Model**

In [133]:
x_train_tensor.shape

torch.Size([455, 30])

In [134]:
# creating model
model = MySimpleNetwork(x_train_tensor.shape[1])

**PipeLine**

In Loop

  - forward pass (The calculation of z = w*x+b => activation function on z)
  - loss calculate
  - backward pass
  - parameter update

We need to define optimizer after creating Models.

We can clear derivates(gradients) anytime that is before backpropogation and after caculating new weights.

**But it is suggested that we should clear it before backprops.**

In [135]:
# Optimizer Stocastic Gradient Descent
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [136]:
for epoch in range(epochs):

  # Forward Pass
  y_pred = model.forward(x_train_tensor)

  # Loss Calculate
  loss = loss_function(y_pred.squeeze(), y_train_tensor)

  # Make gradeints zero(Doing it before backward pass bcoz it is suggested.)
  optimizer.zero_grad()

  # BackWard Loss(backPropogate)
  loss.backward()

  # Update weight & bias using optimizer
  optimizer.step()

  # Print the Loss
  print(f"Epoch:{epoch +1}, Loss:{loss}")
  print('='*60)


Epoch:1, Loss:0.9081440567970276
Epoch:2, Loss:0.6412769556045532
Epoch:3, Loss:0.5084127187728882
Epoch:4, Loss:0.433599054813385
Epoch:5, Loss:0.3852674067020416
Epoch:6, Loss:0.3509739935398102
Epoch:7, Loss:0.325054794549942
Epoch:8, Loss:0.3045726716518402
Epoch:9, Loss:0.2878468930721283
Epoch:10, Loss:0.27384161949157715
Epoch:11, Loss:0.2618808448314667
Epoch:12, Loss:0.25150322914123535
Epoch:13, Loss:0.24238170683383942
Epoch:14, Loss:0.23427750170230865
Epoch:15, Loss:0.22701142728328705
Epoch:16, Loss:0.2204461544752121
Epoch:17, Loss:0.21447448432445526
Epoch:18, Loss:0.2090110331773758
Epoch:19, Loss:0.20398712158203125
Epoch:20, Loss:0.19934648275375366
Epoch:21, Loss:0.1950426995754242
Epoch:22, Loss:0.19103696942329407
Epoch:23, Loss:0.1872965395450592
Epoch:24, Loss:0.1837935894727707
Epoch:25, Loss:0.18050433695316315


In [137]:
"""
This is old code before optimizers. Just here for reference that how much we are cleaning.
"""

# for epoch in range(epochs):

#   # Forward Pass
#   y_pred = model.forward(x_train_tensor)

#   # Loss Calculate
#   loss = loss_function(y_pred.squeeze(), y_train_tensor)

#   # BackWard Loss(backPropogate)
#   loss.backward()

#   # Update the weights (without derivative tracking)
#   # w_new = w_old - (learning_rate * dL/dw) ==> formula
#   # b_new = b_old - (learning_rate * dL/db) ==> formula
#   with torch.no_grad():
#     """
#     Now we cannot directly access the weight.
#     Because we have define linear layer so we have to acces through it.
#     """
#     model.linear.weight -= learning_rate * model.linear.weight.grad
#     model.linear.bias -= learning_rate * model.linear.bias.grad

#   # Clear the gradients for future updates of weights
#   model.linear.weight.grad.zero_()
#   model.linear.bias.grad.zero_()

#   # Print the Loss
#   print(f"Epoch:{epoch +1}, Loss:{loss}")
#   print('='*60)


'\nThis is old code before optimizers. Just here for reference that how much we are cleaning.\n'

The error for dont have same shape in loss function.
The error is because we are calculating loss function on two things y_pred and y_train. But dimension of both are not same.

In [138]:
y_train_tensor.shape

torch.Size([455])

In [139]:
y_pred.shape

torch.Size([455, 1])

So we have to do something to handle this.
Either we can squeeze or reshape/ view in PyTorch to perform that.

In [140]:
y_pred.squeeze().shape

torch.Size([455])

In [141]:
model.linear.weight

Parameter containing:
tensor([[ 1.5804e-01,  2.8743e-01,  3.4498e-01,  3.6789e-01,  2.5111e-01,
          4.1122e-02,  1.2933e-01,  4.1076e-01, -1.7538e-02, -1.8970e-01,
          2.8592e-01, -1.2518e-01,  2.9519e-01,  9.5826e-02, -1.4078e-01,
          1.1109e-01,  1.8116e-02,  3.9392e-02, -1.4859e-01,  9.4826e-02,
          2.1308e-01,  1.0174e-01,  1.0498e-01,  1.0933e-01,  6.7579e-02,
          2.2207e-01,  2.0164e-01,  2.2762e-01,  2.0252e-01,  3.2411e-05]],
       requires_grad=True)

In [142]:
model.linear.bias

Parameter containing:
tensor([-0.1108], requires_grad=True)

## **Evaluation**

In [143]:
with torch.no_grad():
  y_pred = model.forward(x_test_tensor)

print(y_pred)

tensor([[0.5035],
        [0.9071],
        [0.3282],
        [0.0173],
        [0.9628],
        [0.8447],
        [0.0238],
        [0.9449],
        [0.9178],
        [0.9818],
        [0.6984],
        [0.1223],
        [0.1013],
        [0.2504],
        [0.1019],
        [0.9016],
        [0.9906],
        [0.9983],
        [0.6401],
        [0.9976],
        [0.9960],
        [0.0093],
        [0.0644],
        [0.1199],
        [0.0981],
        [0.9865],
        [0.0141],
        [0.9355],
        [0.0343],
        [0.9949],
        [0.0197],
        [0.4132],
        [0.5188],
        [0.6008],
        [0.0686],
        [0.0587],
        [0.0426],
        [0.0582],
        [0.8582],
        [0.8741],
        [0.1683],
        [0.1251],
        [0.0570],
        [0.9991],
        [0.1412],
        [0.2762],
        [0.7445],
        [0.3943],
        [0.1123],
        [0.8918],
        [0.3236],
        [0.9779],
        [0.0193],
        [0.9987],
        [0.0413],
        [0

Now our Prediction is in numbers.

So How we are going to compare thos with the testing values? Because our Testing values are 0s and 1s.

So for such cases simply decide the threshold for predicted values to be 0 and 1.

In [144]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(x_test_tensor)

  # Convert values to 1
  y_pred = (y_pred > 0.9).float()

  # how many predictions are correct by comparing with Test
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy}')


Accuracy: 0.5858725905418396
